In [1]:
import os
import sys
sys.path.append("/shared_data0/chaenyk/exlib-dev/src")
import exlib
from exlib.datasets.supernova import *
from datasets import load_dataset

pa.PyExtensionType.set_auto_load(True)
pyarrow_hotfix.uninstall()

/opt/conda/envs/rapids/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [2]:
DATA_DIR = "/shared_data0/chaenyk/supernova/raw_train_with_labels"
dataset = Supernova(data_dir=DATA_DIR)
train_dataset = dataset['train']
test_dataset = dataset['test']

In [3]:
model_path = '/shared_data0/chaenyk/supernova/model.pt'
config_path = "/shared_data0/chaenyk/supernova/configs/config.yml"
model = SupernovaClsModel(model_path, config_path)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.eval()

num labels: 14
Using Fourier PE
classifier dropout: 0.2


SupernovaClsModel(
  (model): InformerForSequenceClassification(
    (encoder): InformerEncoderFourierPE(
      (value_embedding): InformerValueEmbedding(
        (value_projection): Linear(in_features=8, out_features=768, bias=False)
      )
      (embed_positions): MultiDimFourierPE(
        (fourier_to_hidden): Linear(in_features=384, out_features=32, bias=True)
        (hidden_to_embed): Linear(in_features=32, out_features=768, bias=True)
      )
      (layers): ModuleList(
        (0-7): 8 x InformerEncoderLayer(
          (self_attn): InformerProbSparseAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_

In [4]:
test_dataloader = create_test_dataloader_raw(
    config=model.model_config,
    dataset=test_dataset,
    batch_size=256,
    compute_loss=True
)

original dataset size: 792
remove nans dataset size: 792


In [5]:
y_true = []
y_pred = []
for i, batch in enumerate(test_dataloader):
    print(f"processing batch {i}")
    batch = {k: v.to(device) for k, v in batch.items() if k != "objid"}
    with torch.no_grad():
        outputs = model(**batch)
    y_true.extend(batch['labels'].cpu().numpy())
    y_pred.extend(torch.argmax(outputs.logits, dim=2).squeeze().cpu().numpy())
print(f"accuracy: {sum([1 for i, j in zip(y_true, y_pred) if i == j]) / len(y_true)}")

processing batch 0
processing batch 1
processing batch 2
processing batch 3
accuracy: 0.8017676767676768
